# RAG

- HyDE 와 비교하기 위해서 기본 설정으로 검색해본다.

# 0. Setup

In [1]:
!pip -q install -U boto3 awscli langchain pypdf

In [31]:
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.document_loaders import TextLoader
import langchain

In [3]:
profile_name = None
region = 'us-east-1'

In [4]:
# modelId = 'anthropic.claude-instant-v1'
modelId = 'anthropic.claude-v2'

In [5]:
session = boto3.Session(
    profile_name=profile_name,
    region_name=region,
)
bedrock = session.client(service_name='bedrock-runtime')

## Embeddings

- bedrock embedding model - amazon.titan-embed-text-v1
- 일반적으로는 SOTA 인 baai 의 [bge](https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface) 씀.

In [8]:
llm = Bedrock(
    model_id=modelId,
    client=bedrock,
    model_kwargs={
        'max_tokens_to_sample': 1024
    },
)
embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1",
    client=bedrock,
)

In [44]:
langchain.debug = False

# 1. Usage

- AWS IoT Provisioning Whitepaper [다운로드](https://docs.aws.amazon.com/pdfs/whitepapers/latest/device-manufacturing-provisioning/device-manufacturing-provisioning.pdf#device-manufacturing-provisioning)
- 간접적인 질문을 통해 HyDE 사용

In [45]:
loader = PyPDFLoader("iot.pdf")
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = loader.load_and_split(text_splitter=text_splitter)

In [46]:
len(texts)

36

In [47]:
texts[0]

Document(page_content='Device Manufacturing and \nProvisioning with X.509 \nCertiﬁcates in AWS IoT Core\nAWS Whitepaper', metadata={'source': 'iot.pdf', 'page': 0})

In [ ]:
%%time

docsearch = Chroma.from_documents(texts, embeddings)

In [ ]:
retriever = docsearch.as_retriever(search_kwargs={"k": 5})
retriever.search_type

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [ ]:
query = "What is the best way to provision device when I can not put certificates into my devices?"
llm_response = qa_chain(query)